In [ ]:
import numpy as np
import scipy.signal
from multiprocessing import Pool
from enum import Enum

class Direction(Enum):
    N = 0
    NE = 1
    E = 2
    SE = 3
    S = 4
    SW = 5
    W = 6
    NW = 7

transform = {
    'N': (0,-1),
    'NE': (1,-1),
    'E': (1,0),
    'SE': (1,1),
    'S': (0,1),
    'SW': (-1,1),
    'W': (-1,0),
    'NW': (-1,-1)
}

"""def _is_border(kernel):
    if kernel[0][1] and kernel[1][0] and kernel[2][1] and kernel[1][2]:
        return True
    else:
        return False"""

# [[0,1,0],
#  [1,1,1],
#  [0,1,0]]

def _form_boundary_mask(mask):
    #padded_mask = np.pad(mask, [(1,1),(1,1)], mode='constant', constant_values=[(1,1),(1,1)])
    #padded_mask_shape = padded_mask.shape
    #boundary_mask = np.zeros(mask.shape,dtype=np.bool_)
    kernel = [[0,1,0],
              [1,5,1],
              [0,1,0]]
    boundary_mask = scipy.signal.convolve2d(mask,kernel,boundary="fill",fillvalue=0.0)
    boundary_mask = (boundary_mask < 9.0) & (boundary_mask > 5.0)
    
    return boundary_mask

def _transform_dir(direction):
    return transform[Direction(direction).name]

def _find_next_dir(boundary_mask,cur_pixel,last_dir):
    next_dir = Direction((Direction[last_dir].value-3)%8).name
    inv_dir = Direction((Direction[last_dir].value-4)%8).name
    
    while True:
        next_pixel = [sum(x) for x in zip(cur_pixel,transform[next_dir])]
        if next_dir == inv_dir:
            return None
        elif boundary_mask[next_pixel[1]][next_pixel[0]] == 1:
            return next_pixel,next_dir
        else:
            next_dir = Direction((Direction[next_dir].value+1)%8).name
    
        

def _find_highest_pixel(boundary):
    if len(np.where(boundary == 1)[0]) > 0:
        y = min(np.where(boundary == 1)[0])
        x = min(np.where(boundary[y] == 1)[0])
        return (x,y)
    else:
        return None
    

def _order_boundary_pixels(boundary):    
    ordered_pixels = []
    
    exists_bounds = True

    cur_pixel = _find_highest_pixel(boundary)
    plt.figure(figsize=(20, 20))
    plt.imshow(boundary,cmap='gray')
    plt.show()
    cv2.imwrite("../boundary.png",boundary*255)
    
    while exists_bounds:
        cur_list = []
        
        cur_list.append(cur_pixel)
        last_dir = "E"
        exists_next = True 

        while exists_next:
            boundary[cur_pixel[1],cur_pixel[0]] = 0.0

            res = _find_next_dir(boundary,cur_pixel,last_dir)
            if not res:
                exists_next = False
            else:
                cur_pixel,last_dir = res
                cur_list.append(cur_pixel)        
        
        ordered_pixels.append(cur_list)
        cur_pixel = _find_highest_pixel(boundary)
        if not cur_pixel:
            exists_bounds = False
    
    return ordered_pixels

def gen_sliding_window(img):
    boundary_mask = _form_boundary_mask(img)
    temp_mask = boundary_mask.copy()
    sliding_window = _order_boundary_pixels(temp_mask)
    
    for group in sliding_window:
        count = 0
        
        for pixel in group[::32]:
            print(pixel)
            square = boundary_mask[pixel[1]-31:pixel[1]+32,pixel[0]-31:pixel[0]+32]
            plt.figure(figsize=(4, 4))
            plt.imshow(square,cmap='gray')
            plt.show()
            

    return sliding_window

In [ ]:
import cv2
import matplotlib.pyplot as plt


img = cv2.imread("../img.png",cv2.IMREAD_GRAYSCALE)
#img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(img.shape)
plt.figure(figsize=(20, 20))
plt.imshow(img,cmap='gray')
plt.show()
windows = gen_sliding_window(img/255)
